In [1]:
import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from scipy import stats

In [2]:
total_df=pd.read_csv("risk_data/Train_data_set.csv", encoding='utf-8-sig')

In [3]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,0,126690,0,-1.8,0.0,7.7
1,1,1,0,126690,10,-1.8,0.0,7.7
2,1,1,0,126690,20,-1.8,0.0,7.7
3,1,1,0,126690,30,-1.8,0.0,7.7
4,1,1,0,126690,40,-1.8,0.0,7.7


### pycaret을 이용하여 머신러닝 모델 선정
    1. Data가 커서 최적 모델 선정에 있어 많은 시간이 걸려 보류

In [6]:
from pycaret.classification import *
from time import time

In [7]:
total_df['위험도']=total_df['위험도'].astype('str')

In [8]:
cell_start_time = time()
clf = setup(data = total_df, target='위험도', silent=True)
cell_end_time = time()
print("CELL RUN TIME : ",cell_end_time - cell_start_time)

,Description,Value
0,session_id,1666
1,Target,위험도
2,Target Type,Multiclass
3,Label Encoded,"0.0: 0, 0.3: 1, 0.6: 2, 0.9: 3, 1.2: 4, 1.4: 5, 1.7: 6, 10.0: 7, 2.0: 8, 2.1: 9, 2.3: 10, 2.5: 11, 2.6: 12, 2.7: 13, 2.8: 14, 2.9: 15, 3.0: 16, 3.1: 17, 3.2: 18, 3.3: 19, 3.4: 20, 3.5: 21, 3.7: 22, 3.8: 23, 3.9: 24, 4.1: 25, 4.2: 26, 4.4: 27, 4.5: 28, 4.6: 29, 4.7: 30, 4.8: 31, 5.0: 32, 5.1: 33, 5.2: 34, 5.3: 35, 5.4: 36, 5.5: 37, 5.6: 38, 5.7: 39, 5.8: 40, 5.9: 41, 6.0: 42, 6.2: 43, 6.3: 44, 6.4: 45, 6.6: 46, 6.7: 47, 6.9: 48, 7.0: 49, 7.1: 50, 7.2: 51, 7.3: 52, 7.5: 53, 7.6: 54, 7.7: 55, 7.9: 56, 8.0: 57, 8.2: 58, 8.3: 59, 8.5: 60, 8.8: 61, 9.1: 62, 9.4: 63, 9.7: 64"
4,Original Data,"(2448189, 8)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


CELL RUN TIME :  23.554945945739746


In [4]:
# 모델 성능 비교  # 잠정 보류
cell_start_time = time()
top5_models = compare_models(fold=5, round=3, sort = 'Accuracy', n_select = 5)
cell_end_time = time()
print("CELL RUN TIME : ",cell_end_time - cell_start_time)

NameError: name 'time' is not defined

In [ ]:
top5_models

### RandomForest을 이용한 데이터 평가

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

In [5]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [6]:
train_data.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량
0,1,1,0,126690,0,-1.8,0.0
1,1,1,0,126690,10,-1.8,0.0
2,1,1,0,126690,20,-1.8,0.0
3,1,1,0,126690,30,-1.8,0.0
4,1,1,0,126690,40,-1.8,0.0


In [7]:
target_data.head()

,위험도
0,7.7
1,7.7
2,7.7
3,7.7
4,7.7


In [8]:
# target_data one-hot-encoding
target_data2 = pd.get_dummies(target_data['위험도'])
target_data_result = pd.concat([target_data, target_data2], axis=1)

In [9]:
target_data2.head()

,0.0,0.3,0.6,0.7,1.0,1.2,1.4,1.5,1.6,1.7,...,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
train_x, test_x, train_y, test_y = train_test_split(train_data, target_data2, test_size = 0.2, random_state = 42)

In [11]:
clf = RandomForestClassifier(random_state=42)
clf.fit(train_x,train_y)

RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42)

In [12]:
predict1 = clf.predict(test_x)
print(accuracy_score(test_y,predict1))

0.3620095225777246


In [13]:
# 모델 저장
import joblib
joblib.dump(clf, 'model/Randomforest.pkl')

['model/Randomforest.pkl']

In [14]:
# 모델 불러오기
rf_model=joblib.load('model/Randomforest.pkl')

In [15]:
predict1 = rf_model.predict(test_x)
print(accuracy_score(test_y,predict1))

0.3620095225777246


### XGBoost Classifier 모델 이용

In [16]:
import xgboost

In [17]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [18]:
train_X, test_X, train_Y, test_Y = train_test_split(train_data, target_data, test_size = 0.2, random_state = 42)

In [19]:
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

(1979292, 7) (494824, 7) (1979292, 1) (494824, 1)


In [29]:
model = xgboost.XGBClassifier()
model.fit(train_X.values, np.ravel(train_Y.values))

[21:33:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [37]:
test_X.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량
340055,2,16,188,325603,0,3.1,2.4
226028,2,2,133,2381390,30,-1.9,0.0
366176,2,20,26,311381,110,5.6,0.6
2351197,12,13,99,770420,120,2.2,1.1
905599,5,2,165,0,60,17.4,5.0


In [40]:
train_Y

,위험도
460215,7.3
1950589,2.5
2395716,2.1
670213,7.6
1474059,0.0
...,...
110268,1.5
1692743,3.7
2356330,4.1
2229084,5.0


In [31]:
test_X.values

array([[ 2.00e+00,  1.60e+01,  1.88e+02, ...,  0.00e+00,  3.10e+00,
         2.40e+00],
       [ 2.00e+00,  2.00e+00,  1.33e+02, ...,  3.00e+01, -1.90e+00,
         0.00e+00],
       [ 2.00e+00,  2.00e+01,  2.60e+01, ...,  1.10e+02,  5.60e+00,
         6.00e-01],
       ...,
       [ 3.00e+00,  1.10e+01,  1.58e+02, ...,  5.00e+01,  5.40e+00,
         6.00e-01],
       [ 5.00e+00,  7.00e+00,  2.80e+01, ...,  7.00e+01,  1.70e+01,
         4.70e+00],
       [ 1.00e+01,  1.80e+01,  9.10e+01, ...,  2.50e+02,  1.51e+01,
         2.00e-01]])

In [32]:
y_pred = model.predict(test_X.values)
#predictions = [np.round(value,1) for value in y_pred]

In [33]:
print("(pred == y_test) score: {0:.3f}".format((y_pred==test_Y['위험도'].values).mean()))

(pred == y_test) score: 0.731


In [34]:
# 모델 저장
import joblib
joblib.dump(model, 'model/xgboost.pkl')

['model/xgboost.pkl']

In [35]:
# 모델 불러오기
xgb_model=joblib.load('model/xgboost.pkl')

In [41]:
y_pred = xgb_model.predict(test_X.values)
#predictions = [np.round(value,1) for value in y_pred]

In [42]:
print("(pred == y_test) score: {0:.3f}".format((y_pred==test_Y['위험도'].values).mean()))

(pred == y_test) score: 0.731


In [43]:
y_pred

array(['5.6', '6.5', '3.9', ..., '7.1', '6.5', '0.0'], dtype=object)

In [44]:
test_Y['위험도'].values

array(['5.6', '6.5', '3.9', ..., '9.6', '6.5', '0.0'], dtype=object)

### SVM 모델 적용

In [5]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [6]:
train_data.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량
0,1,1,0,126690,0,-1.8,0.0
1,1,1,0,126690,10,-1.8,0.0
2,1,1,0,126690,20,-1.8,0.0
3,1,1,0,126690,30,-1.8,0.0
4,1,1,0,126690,40,-1.8,0.0


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
scaler.fit(train_data)
X_scaled = scaler.transform(train_data)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, target_data, test_size = 0.2, random_state = 100)

In [10]:
import sklearn.svm as svm
from sklearn.model_selection import cross_val_score, cross_validate

In [11]:
svm_clf =svm.SVC(kernel = 'rbf', random_state=100)

In [12]:
from sklearn.model_selection import GridSearchCV

In [13]:
parameters = {'C': [0.001, 0.01, 0.1],
             'gamma':[0.001, 0.01, 0.1]}

In [14]:
grid_svm = GridSearchCV(svm_clf,
                      param_grid = parameters, cv = 5)

In [ ]:
grid_svm.fit(X_train, np.ravel(Y_train))

In [ ]:
result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)